### Setup

In [4]:
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import string
import os

from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional

### Get the dataset

In [28]:
songdata = pd.read_csv("/home/login/Documents/songdata.csv", dtype = str)[:250]
songdata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250 entries, 0 to 249
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   artist  250 non-null    object
 1   song    250 non-null    object
 2   link    250 non-null    object
 3   text    250 non-null    object
dtypes: object(4)
memory usage: 7.9+ KB


### 250 songs


### Preprocessing

In [29]:
def tokenize_corpus(corpus, num_words = -1):
    if num_words > -1:
        tokenizer = Tokenizer(num_words = num_words)
    else:
        tokenizer = Tokenizer()
    tokenizer.fit_on_texts(corpus)
    return tokenizer

def create_lyrics_corpus(dataset, field):
    dataset[field] = dataset[field].str.replace("[{}]".format(string.punctuation), "")
    dataset[field] = dataset[field].str.lower()
    lyrics = dataset[field].str.cat()
    corpus = lyrics.split("\n")
    
    for _ in range(len(corpus)):
        corpus[_] = corpus[_].rstrip()
        
    corpus = [_ for _ in corpus if _ != ""]
    return corpus

In [30]:
corpus = create_lyrics_corpus(songdata, "text")
tokenizer = tokenize_corpus(corpus)

total_words = len(tokenizer.word_index) + 1
print(total_words)

4151


### Create sequences

In [62]:
sequences = []
i = 0
for line in corpus:
    tok_line = tokenizer.texts_to_sequences([line])[0]
    for _ in range(1, len(tok_line)):
        line_sequences = tok_line[:_+1]
        sequences.append(line_sequences)
        
maxLength = max([len(_) for _ in sequences])
padded_sequences = pad_sequences(sequences, maxlen = maxLength, padding = "pre")
print(maxLength)
print(padded_sequences[0, :])

20
[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
 112  65]


### Create labels

In [72]:
inputs, labels = padded_sequences[:, :-1], padded_sequences[:, -1]
one_hot_labels = tf.keras.utils.to_categorical(labels, num_classes = total_words)
print(inputs[0])
print(labels[0])
print(one_hot_labels[0])
print(one_hot_labels[1])

[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
 112]
65
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]


### Train a Better Text Generation Model

In [73]:
embedding_dim = 64
model = Sequential([
    Embedding(total_words, embedding_dim, input_length = maxLength - 1),
    Bidirectional(LSTM(20)),
    Dense(total_words, activation = "softmax")
])
model.compile(loss = tf.keras.losses.CategoricalCrossEntropy(),
             optimizer = tf.keras.optimizers.Adam(),
             metrics = ['accuracy'])
model.

AttributeError: module 'keras.api._v2.keras.losses' has no attribute 'CategoricalCrossEntropy'